# metatrop_metastatic_sites_combined.py

### By Chris Fong - MSKCC 2020

This script will create a standardized table of metastatic sites for for a given 
- Diagnosis table containing ICD billing codes and 
- Clinical sample table downloaded from the GENIE V9 dataset 

The script can compute if a distant LN metastasis occurred for that patient, given their cancer type according to IMPACT data. Similarly, local extension is computed for viseral organs.

If a patient has multiple dx from IMPACT, more than 1 row will exist for that patient, and should be removed.



## Load Libraries

In [3]:
import sys  
sys.path.insert(0, '../mappings')
sys.path.insert(0, '../analysis')
sys.path.insert(0, '../')
import os
import pandas as pd
import constants_o_sites as const
from organ_mapping_analysis import OrganMappingAnalysisRND
from organ_mapping_rdn_processing import MetastaticSpreadMappingRND

In [4]:
# Console settings
def set_debug_console():
    desired_width = 320
    pd.set_option('display.width', desired_width)
    pd.set_option('display.max_rows', 250)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    pd.set_option('display.expand_frame_repr', False)

set_debug_console()

## Load Data

##### Define columns being used

In [6]:
## Load Data
fname_save_anno = 'metatrop_met_site_annotations_clinical_and_impact_20201021.csv'
path_save = '/Users/fongc2/Documents/github/MSK/clinical_data_mining/organ-site-mapping/metastatic_tropism_work'
pathfilename_save = os.path.join(path, fname_save_anno)

# File for ICD billing
col_icd_billing = 'ICD-9/10 Dx Code'
col_id = 'SAMPLE_ID'
col_id2 = 'DMP_ID'
# File for cBioPortal data
col_ct = 'CANCER_TYPE'
col_prim_site = 'PRIMARY_SITE'
col_met_site = 'METASTATIC_SITE'
col_sample_type = 'SAMPLE_TYPE'
col_sex = 'SEX'

##### Load GENIE data from cbioportal 

In [42]:
fname = 'genie_private_clinical_data.tsv'
path = '/Users/fongc2/Documents/github/MSK/DARWIN_ETL/data/'
pathfilename1 = os.path.join(path, fname)
df_samples1 = pd.read_csv(pathfilename1, header=0, low_memory=False, sep='\t')

##### Load IMPACT data from cbioportal to obtain primary and metastatic site info

In [43]:
fname = 'mskimpact_clinical_data.tsv'
path = '/Users/fongc2/Documents/github/MSK/DARWIN_ETL/data/'
pathfilename1 = os.path.join(path, fname)
df_impact = pd.read_csv(pathfilename1, header=0, low_memory=False, sep='\t')

##### Load ICD Billing Patient Diagnosis Data 

In [55]:
# Load met sites directly from diagnosis
fname = 'table_diagnosis_clean.csv'
path = '/Users/fongc2/Documents/github/MSK/DARWIN_ETL/data/'
pathfilename1 = os.path.join(path, fname)
df_dx = pd.read_csv(pathfilename1, header=0, low_memory=False, sep=',')

In [99]:
list(df_dx.columns)

['DMP_ID',
 'P_ID',
 'ICD-9/10 Dx Code',
 'ICD-O Site Code',
 'ICD-O Site Desc',
 'ICD-O Histology Code',
 'ICD-O Histology Desc',
 'Clinical Group',
 'Path Group',
 'AJCC',
 'Summary',
 'MSK Stage',
 'Diagnosis Type',
 'CATEGORY1',
 'CATEGORY2',
 'CATEGORY3',
 'CATEGORY4',
 'Source',
 'DX_DESCRIPTION',
 'Diagnosis Date',
 'REASON_NO_DATA',
 'DATA_AVAILABLE_DDP',
 'IS_MET_ICD_BILLING',
 'IS_MET_ICD_BILLING_OTHER',
 'IS_MET_ICD_BILLING_LN',
 'IS_MET_ICD_BILLING_NON_LN',
 'IS_NON_CANCER',
 'IS_INVALID_LABEL',
 'IS_ICDO_DX',
 'IS_STAGE_IV_ICDO',
 'ICDO_IS_LOCAL',
 'ICDO_IS_REGIONAL_GENERAL',
 'ICDO_IS_DISTANT',
 'ICDO_IS_IN_SITU',
 'ICDO_IS_REGIONAL_LYMPH',
 'ICDO_IS_REGIONAL_DIRECT_EXT',
 'ICDO_IS_UNSTAGED',
 'ICDO_IS_STAGE_NA',
 'IS_MALIGNANT_PRIMARY']

## Clean Data

### Clean clinical sample file from cbioportal

##### Fix header lables

In [44]:
# Headers to replace
col_rep = {'Patient ID': col_id2,
           'Sample ID': col_id,
           'Sex': col_sex,
           'Cancer Type': col_ct,
           'Primary Tumor Site': col_prim_site,
           'Metastatic Site': col_met_site,
           'Sample Type': col_sample_type}

In [45]:
# Genie data
df_samples1 = df_samples1.rename(columns=col_rep)
cols_genie = set.intersection(set(list(col_rep.values())), set(df_samples1.columns))
cols_genie = [x for x in df_samples1.columns if x in cols_genie]
df_samples = df_samples1[cols_genie].copy()
df_samples[col_id] = df_samples[col_id].str.replace('GENIE-MSK-', '')
df_samples[col_id2] = df_samples[col_id2].str.replace('GENIE-MSK-', '')

In [66]:
df_samples.head(2)

,DMP_ID,SAMPLE_ID,CANCER_TYPE,SAMPLE_TYPE,SEX
0,P-0000004,P-0000004-T01-IM3,Breast Cancer,Primary,Female
1,P-0000012,P-0000012-T02-IM3,Breast Cancer,Primary,Female


In [67]:
# IMPACT data
df_impact1 = df_impact.rename(columns=col_rep)
df_samples_impact = df_impact1[[col_id, col_prim_site, col_met_site]]

In [72]:
df_samples_impact.head()

,SAMPLE_ID,PRIMARY_SITE,METASTATIC_SITE
0,P-0000004-T01-IM3,Breast,NaN
1,P-0000012-T02-IM3,Breast,NaN
2,P-0000012-T03-IM3,Lung,Neck
3,P-0000012-T04-IM6,Lung,NaN
4,P-0000015-T01-IM3,Breast,Liver


##### Merge GENIE and IMPACT data sets

In [73]:
df_samples_f = df_samples.merge(right=df_samples_impact, how='left', on='SAMPLE_ID')

##### Create dataframe for metastatic sites only from clinical sample file

In [77]:
df_metatrop_met = df_samples_f[df_samples_f[col_sample_type] == 'Metastasis']


In [78]:
df_metatrop_met.head()

,DMP_ID,SAMPLE_ID,CANCER_TYPE,SAMPLE_TYPE,SEX,PRIMARY_SITE,METASTATIC_SITE
2,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,Metastasis,Female,Lung,Neck
4,P-0000015,P-0000015-T01-IM3,Breast Cancer,Metastasis,Female,Breast,Liver
6,P-0000024,P-0000024-T01-IM3,Endometrial Cancer,Metastasis,Female,Uterus,Lung
8,P-0000025,P-0000025-T02-IM5,Endometrial Cancer,Metastasis,Female,Uterus,Peritoneum
9,P-0000025,P-0000025-T03-IM6,Endometrial Cancer,Metastasis,Female,Uterus,Liver


The table above is now in the required format for mapping metastatic sites from IMPACT free text notations to a standard naming convention.

### Clean clinical diagnosis data from Darwin

##### Filter by metastatic ICD billing codes and select relevant columns

In [79]:
cols_select = [col_id2, col_icd_billing, 'DX_DESCRIPTION', 'IS_MET_ICD_BILLING']
df_dx_mets1 = df_dx.loc[df_dx['IS_MET_ICD_BILLING'] == True, cols_select]


##### Merge with info on gender/sex

In [88]:
df_dx_mets = df_dx_mets1.merge(right=df_samples_f[[col_id2, col_sex, col_prim_site, col_met_site]], how='left', on=col_id2)
df_dx_mets.head()

,DMP_ID,ICD-9/10 Dx Code,DX_DESCRIPTION,IS_MET_ICD_BILLING,SEX,PRIMARY_SITE,METASTATIC_SITE
0,P-0000004,197.7,['197.7 - LIVER METS'],True,Female,Breast,NaN
1,P-0000004,198.5,['198.5 - BONE METS'],True,Female,Breast,NaN
2,P-0000012,196.0,['196.0 - LYMPH NODE METS-NECK'],True,Female,Breast,NaN
3,P-0000012,196.0,['196.0 - LYMPH NODE METS-NECK'],True,Female,Lung,Neck
4,P-0000012,196.0,['196.0 - LYMPH NODE METS-NECK'],True,Female,Lung,NaN


## Map IMPACT and Darwin Clinical Diagnoses Data to Standard Organ Sites

### Load Mapping Objects

##### Load mapping tables

In [85]:
## Load Mapping Objects### Load mapping tables# Load RDN mapping
obj_met_map_rdn = MetastaticSpreadMappingRND(path='../' + const.pathname,
                                             fname_all_sites=const.fname_mapping_rdn_all_sites,
                                             fname_hematogenous=const.fname_mapping_rdn_hematogenous,
                                             fname_localext=const.fname_mapping_rdn_localext,
                                             fname_lymphatic=const.fname_mapping_rdn_lymphatic,
                                             fname_site_map=const.fname_mapping_rdn_site_map,
                                             fname_billing_map=const.fname_mapping_rdn_billing_map,
                                             fname_billing_code_dict=const.fname_mapping_rdn_to_billing_codes)

Loading mapping tables


##### Load mapping object

In [86]:
### Load mapping object# Load annoations object
obj_mapping = OrganMappingAnalysisRND(obj_met_map=obj_met_map_rdn)

### Run mapping objects

##### Create Mapping based on Clinical Sample Table (Primary Site) and ICD Billing Codes (Metastatic Site)

In [101]:
# Annotate diagnosis table of ICD billings with renzo's mapping of metastatic sites
df_met_sites_dx = obj_mapping.annotate_icd_billing_met_dx(df_dx_mets=df_dx_mets, col_icd_billing=col_icd_billing, col_sex=col_sex)


df_met_rdn_anno = obj_mapping.create_sample_to_icd_billing_met_mapping(df_dx=df_dx_mets,
                                                                       col_icd_billing=col_icd_billing,
                                                                       col_sex=col_sex,
                                                                       df_samples=df_samples_f,
                                                                       col_primary_site=col_prim_site,
                                                                       col_met_site=col_met_site,
                                                                       label_dist_ln=True)

In [103]:
df_met_rdn_anno.head(3)

,SAMPLE_ID,DMP_ID,CANCER_TYPE,PRIMARY_SITE_x,METASTATIC_SITE_x,SEX,PRIMARY_SITE_RDN_MAP,ICD-9/10 Dx Code,DX_DESCRIPTION,IS_MET_ICD_BILLING,PRIMARY_SITE_y,METASTATIC_SITE_y,METASTATIC_SITE_RDN_MAP,LYMPH_SPREAD,LOCAL_EXTENSION,hematogenous_grouping,METASTATIC_SITE_ONCOTREE_RDN,METASTATIC_SITE_BILLING_RDN
0,P-0000004-T01-IM3,P-0000004,Breast Cancer,Breast,NaN,Female,chest_breast,197.7,['197.7 - LIVER METS'],True,Breast,NaN,abdomen_liver,NaN,NaN,LIVER,Liver,LIVER
1,P-0000004-T01-IM3,P-0000004,Breast Cancer,Breast,NaN,Female,chest_breast,198.5,['198.5 - BONE METS'],True,Breast,NaN,undeterminate_bone,NaN,NaN,NON_PORTAL,Bone,BONE
2,P-0000012-T02-IM3,P-0000012,Breast Cancer,Breast,NaN,Female,chest_breast,196.0,['196.0 - LYMPH NODE METS-NECK'],True,Breast,NaN,neck_node,DISTANT,NaN,NON_PORTAL,Distant Lymphatic,DIST_LYMPH


##### Create Mapping on GENIE's Clinical Samples Table from cBioPortal

In [105]:
# Annotate IMPACT sample site data
df_met_sites_impact = obj_mapping.annotate_mapping_impact_met_samples(df_samples=df_metatrop_met, 
                                                                      col_primary_site=col_prim_site, 
                                                                      col_met_site=col_met_site, 
                                                                      label_dist_ln=True)

df_met_sites_impact.head(3)

,DMP_ID,SAMPLE_ID,CANCER_TYPE,SAMPLE_TYPE,SEX,PRIMARY_SITE,METASTATIC_SITE,PRIMARY_SITE_RDN_MAP,PRIMARY_SITE_RDN_MAP_MAIN,PRIMARY_SITE_RDN_MAP_SECONDARY,METASTATIC_SITE_RDN_MAP,METASTATIC_SITE_RDN_MAP_MAIN,METASTATIC_SITE_RDN_MAP_SECONDARY,LYMPH_SPREAD,LOCAL_EXTENSION,hematogenous_grouping,METASTATIC_SITE_ONCOTREE_RDN,METASTATIC_SITE_BILLING_RDN
0,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,Metastasis,Female,Lung,Neck,chest_lung,chest,lung,neck_unknown,neck,unknown,NaN,NaN,NON_PORTAL,Head and Neck,HEAD_AND_NECK
1,P-0000015,P-0000015-T01-IM3,Breast Cancer,Metastasis,Female,Breast,Liver,chest_breast,chest,breast,abdomen_liver,abdomen,liver,NaN,NaN,LIVER,Liver,LIVER
2,P-0000024,P-0000024-T01-IM3,Endometrial Cancer,Metastasis,Female,Uterus,Lung,pelvis_uterus,pelvis,uterus,chest_lung,chest,lung,NaN,NaN,LUNG,Lung,LUNG


### Combine mapping from ICD billing and IMPACT clinical samples

In [93]:
cols_keep1 = list(set.intersection(set(df_met_rdn_anno.columns), set(df_met_sites_impact.columns)))

cols_keep = [x for x in list(df_met_rdn_anno.columns) if x in cols_keep1]
cols_keep

['SAMPLE_ID',
 'DMP_ID',
 'CANCER_TYPE',
 'SEX',
 'PRIMARY_SITE_RDN_MAP',
 'METASTATIC_SITE_RDN_MAP',
 'LYMPH_SPREAD',
 'LOCAL_EXTENSION',
 'hematogenous_grouping',
 'METASTATIC_SITE_ONCOTREE_RDN',
 'METASTATIC_SITE_BILLING_RDN']

In [94]:
# Subset dataframes
df_met_sites_impact2 = df_met_sites_impact[cols_keep]
df_met_rdn_anno1 = df_met_rdn_anno[cols_keep]

# Add source annotation
df_met_sites_impact2 = df_met_sites_impact2.assign(SOURCE='Pathology')
df_met_rdn_anno1 = df_met_rdn_anno1.assign(SOURCE='ICD Billing')

df_f = pd.concat([df_met_sites_impact2, df_met_rdn_anno1], axis=0, sort=False)

df_f.head(50)

,SAMPLE_ID,DMP_ID,CANCER_TYPE,SEX,PRIMARY_SITE_RDN_MAP,METASTATIC_SITE_RDN_MAP,LYMPH_SPREAD,LOCAL_EXTENSION,hematogenous_grouping,METASTATIC_SITE_ONCOTREE_RDN,METASTATIC_SITE_BILLING_RDN,SOURCE
0,P-0000012-T03-IM3,P-0000012,Non-Small Cell Lung Cancer,Female,chest_lung,neck_unknown,NaN,NaN,NON_PORTAL,Head and Neck,HEAD_AND_NECK,Pathology
1,P-0000015-T01-IM3,P-0000015,Breast Cancer,Female,chest_breast,abdomen_liver,NaN,NaN,LIVER,Liver,LIVER,Pathology
2,P-0000024-T01-IM3,P-0000024,Endometrial Cancer,Female,pelvis_uterus,chest_lung,NaN,NaN,LUNG,Lung,LUNG,Pathology
3,P-0000025-T02-IM5,P-0000025,Endometrial Cancer,Female,pelvis_uterus,abdomen_peritoneum.serosa,NaN,LOCAL,PORTAL,Peritoneum,PERITONEUM,Pathology
4,P-0000025-T03-IM6,P-0000025,Endometrial Cancer,Female,pelvis_uterus,abdomen_liver,NaN,NaN,LIVER,Liver,LIVER,Pathology
5,P-0000026-T01-IM3,P-0000026,Endometrial Cancer,Female,pelvis_uterus,pelvis_unknown,NaN,NaN,NON_PORTAL,NaN,OTHER,Pathology
6,P-0000030-T01-IM3,P-0000030,Non-Small Cell Lung Cancer,Female,chest_lung,undeterminate_node,NaN,NaN,NON_PORTAL,Lymphatic,LYMPH,Pathology
7,P-0000037-T01-IM3,P-0000037,Hepatobiliary Cancer,Male,abdomen_liver,abdomen_liver,NaN,LOCAL,LIVER,Liver,LIVER,Pathology
8,P-0000043-T02-IM3,P-0000043,Bladder Cancer,Unknown,pelvis_bladder,abdomen_liver,NaN,NaN,LIVER,Liver,LIVER,Pathology
9,P-0000058-T01-IM3,P-0000058,Breast Cancer,Female,chest_breast,abdomen_liver,NaN,NaN,LIVER,Liver,LIVER,Pathology


## Save annotations

In [95]:
## Save annotations
df_f.to_csv(fname_save_anno, index=False)

## Create Binary Matrix from Mapping

In [96]:
df_binary = obj_mapping.create_binary_met_sites(df=df_f, col_index='SAMPLE_ID', col_count='DMP_ID', col_met_site='METASTATIC_SITE_BILLING_RDN')


In [97]:
df_binary.head(100)

METASTATIC_SITE_BILLING_RDN,SAMPLE_ID,HAS_MET_ADRENAL_GLAND,HAS_MET_BILIARY_TRACT,HAS_MET_BLADDER_OR_URINARY_TRACT,HAS_MET_BONE,HAS_MET_BOWEL,HAS_MET_BREAST,HAS_MET_CNS_BRAIN,HAS_MET_DIST_LYMPH,HAS_MET_GENITAL_FEMALE,HAS_MET_GENITAL_MALE,HAS_MET_HEAD_AND_NECK,HAS_MET_KIDNEY,HAS_MET_LIVER,HAS_MET_LUNG,HAS_MET_LYMPH,HAS_MET_MEDIASTINUM,HAS_MET_OTHER,HAS_MET_OVARY,HAS_MET_PERIPHERAL_NERVOUS_SYSTEM,HAS_MET_PERITONEUM,HAS_MET_PLEURA,HAS_MET_REGIONAL_LYMPH,HAS_MET_SKIN
0,P-0000004-T01-IM3,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,P-0000012-T02-IM3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
2,P-0000012-T03-IM3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0
3,P-0000012-T04-IM6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
4,P-0000015-T01-IM3,0,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0
5,P-0000023-T01-IM3,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0
6,P-0000024-T01-IM3,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0
7,P-0000025-T01-IM3,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0
8,P-0000025-T02-IM5,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0
9,P-0000025-T03-IM6,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0
